In [1]:
from Librerias_SAP import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

In [2]:
def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""--------------------------------FUNCION PARA LIMPIAR NUMEROS A FLOAT--------------------------------------------------------"""

def Clean_column_number(column):
    column=column.apply(lambda x: float(str(x).strip().replace(",","")))
    return(column)

def default_column(default_columns,dataframe):     #Parametros (default_columns: Columnas predeterminadas,dataframe:)
    diccionay_default_column={}
    if len(default_columns)==len(dataframe.columns):
        for i in range(0,len(default_columns)):
            diccionay_default_column[dataframe.columns[i]]=default_columns[i]   
        dataframe=dataframe.rename(columns=diccionay_default_column)   #Remplazo las columnas con las de default
    else:
        None   #asumo columnas originales como estandar
    return(dataframe)

def Date_Null(dataframe,date):
    filter=(date!="00.00.0000")
    dataframe=dataframe.loc[filter]
    return(dataframe)

In [3]:
""" Search MB52 bulk"""
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][1]
password=Keys["Password"][1]
# Initialize session
Iniciar=SAP_GUI.SessionSAP(user,password)
# Start transsación
Transsacion="mb52"
provision="GRANELES"   
variant="AUXNVOSPT"
Buscar_MB52_GRANELES=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)


In [4]:
Name_GRANELES="GRANELES"  # Name file
SAP_GUI.Export_TXT2(Name_GRANELES,Iniciar)

In [5]:
Ruta_GRANELES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_GRANELES)+".txt"
data_GRANELES = pd.read_csv(Ruta_GRANELES, skiprows=1, delimiter='\t')

In [6]:
data_GRANELES=Clean_Columns(data_GRANELES)

In [7]:
default_column_MB52=['Material', 'Texto breve de material', 'Gpo.artíc.', 'Ce.', 'Alm.',
       'Lote', 'UMB', 'Libre utilización', 'En control calidad',
       'Valor libre util.', 'Valor en insp.cal.', 'Cad./FPC']   #Disposicion PROXI A VENCER


In [8]:
data_GRANELES=default_column(default_column_MB52,data_GRANELES) #Grabeles limpiar

In [9]:
data_GRANELES=Date_Null(data_GRANELES,data_GRANELES['Cad./FPC'])

"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_GRANELES=data_GRANELES[data_GRANELES.Material.notnull()]
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].apply(lambda x: datetime.strptime(x,'%d.%m.%Y'))  #Change columns str to datetime



In [10]:
Data_Denominacion=Clean_Columns(pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Denominacion_Articulos.txt",delimiter='\t'))[["Grupo art.","Denominación 2 del gr.artículos"]]
Data_Denominacion=Data_Denominacion.rename(columns={"Denominación 2 del gr.artículos":"Denom.gr.artíc."})
data_GRANELES=pd.merge(Data_Denominacion,data_GRANELES,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")

In [11]:
Data_Denominacion

,Grupo art.,Denom.gr.artíc.
0,S049,Servicios
1,S048,Servicios
2,S047,Servicios
3,C050,Compras
4,B036,Compras
...,...,...
483,R018,Mantenimiento
484,C141,VERP DECORADO RIONEG
485,C100,Compras
486,C027,Wellonda


In [12]:
"""---------------------------------Filtro graneles vencidos-------------------"""
now=datetime.now()
month_vencido=now-timedelta(days=30) 
mask_vencido=(data_GRANELES["Cad./FPC"]<=now) & (data_GRANELES["Cad./FPC"]>=month_vencido)
data_GRANELES_vencidos=data_GRANELES.loc[mask_vencido]
#Sort values
data_GRANELES_vencidos=data_GRANELES_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_GRANELES_vencidos=data_GRANELES_vencidos.reset_index(drop=True)
data_GRANELES_vencidos["Cad./FPC"]=data_GRANELES_vencidos["Cad./FPC"].dt.date   #convet datetime to date 

In [13]:
# filter dates between now a 30 month later
now=datetime.now()
month=timedelta(days=6*30)   # Change limiter day(30 days= 1 month)
month=now+month
mask=(data_GRANELES["Cad./FPC"]>=now) & (data_GRANELES["Cad./FPC"]<=month)
data_GRANELES=data_GRANELES.loc[mask]
#Sort values
data_GRANELES=data_GRANELES.sort_values(by=["Cad./FPC","Libre utilización"])
data_GRANELES=data_GRANELES.reset_index(drop=True)
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].dt.date   #convet datetime to date 

In [18]:
"""------------DATAFRAME GRANALES PARA INFORME-----------------------------------------------------------------------"""

data_Graneles_informe=data_GRANELES[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]


data_Graneles_Vencidos_informe=data_GRANELES_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote', 'UMB',
       'Cad./FPC']]

data_Graneles_Vencidos_informe
data_Graneles_informe

,Material,Texto breve de material,Gpo.artíc.,Denom.gr.artíc.,Ce.,Alm.,Lote,Libre utilización,En control calidad,UMB,Valor libre util.,Valor en insp.cal.,Cad./FPC
0,PB0079251,FRAG. MALE MASS. AMBICIOSO NOVAV,C266,GR.NOVAVENTA FPT,1000.0,1001.0,10890449,20,0,KG,"832,875",0,2023-06-23
1,PB0060364,DEO-AP ROLLON JM FEMENINO FRAG. CLEAN,C220,GR.Jerónimo Martens,1000.0,1001.0,10899920,0,0,KG,0,0,2023-06-24
2,PB0093601,DEO-AP ROLL-ON ORIGINAL AFM,C014,GR. Prebel,1000.0,1001.0,10899908,0,0,KG,0,0,2023-06-24
3,PB0084169,CORRECTOR B.SABILA T3.5,C014,GR. Prebel,1000.0,1001.0,10902327,0,5.150,KG,0,"836,826",2023-06-26
4,PB0085121,LACA NORMAL NATURAL STYLING FULL,C241,GR.HENKEL,1000.0,1001.0,10902325,797,0,KG,"8,633,989",0,2023-06-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,PB0089408,50377954 CX2 AGUAS COLONIA VIOLETA,C042,GR. Natura,1000.0,1001.0,10901326,26,0,KG,"428,136",0,2023-12-06
62,PB0079457,50063072 SOL. COL. ALC. NAT. VERD. PUR.,C042,GR. Natura,1000.0,1001.0,10902110,0.970,0,KG,"6,726",0,2023-12-13
63,PB0023590,"50061043 SOL. COL. ACU. NAT. AMA5(0,05%)",C042,GR. Natura,1000.0,1001.0,10902316,0,"30,000",G,0,"322,198",2023-12-18
64,PB0023586,50063071 SOL. COL. ALC. NAT. NAR(0.1%),C042,GR. Natura,1000.0,1001.0,10902318,"10,000",0,G,"120,030",0,2023-12-18


In [20]:
"""-----------------------------GENERO SUBTOTALES------------------------------------------------------------"""

data_GRANELES_Libre=Clean_column_number(data_GRANELES["Valor libre util."]).sum()
data_GRANELES_Calidad=Clean_column_number(data_GRANELES['Valor en insp.cal.']).sum()
Total_Graneles=data_GRANELES_Libre+data_GRANELES_Calidad
Total_Graneles="${:,.2f}".format(Total_Graneles)

In [21]:
Total_Graneles

'$96,504,171.00'

In [44]:
""" Search MB52 raw material"""

# Start transsación
Transsacion="mb52"
provision="MATERIA PRIMA"  
variant="AUXNVOSPT"
Buscar_MB52_MATERIAS=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)
# Save file



Name_Materias="MATERIAS PRIMAS"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_Materias,Iniciar)
# Pass the route and read file 
Ruta_Materias="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_Materias)+".txt"
data_Materias=pd.read_csv(Ruta_Materias,delimiter="\t")
# Change, drop and standardize columns
data_Materias=Clean_Columns(data_Materias)
data_Materias=default_column(default_column_MB52,data_Materias) 
data_Materias=Date_Null(data_Materias,data_Materias['Cad./FPC'])


"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_Materias=data_Materias[data_Materias.Material.notnull()]
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].apply(lambda x: datetime.strptime(str(x),'%d.%m.%Y'))  #Change columns str to datetime

data_Materias=pd.merge(Data_Denominacion,data_Materias,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


In [25]:
"""---------------------------Buscamos las clases de material------------- """
Data_Material_Total=data_Materias
Lista_tipo_Materias=data_Materias["Gpo.artíc."].drop_duplicates().reset_index(drop=True)


In [26]:
"""--------------------------------------MATERIAS PRIMAS VENCIDAS-------------------------"""

mask_vencido=(data_Materias["Cad./FPC"]<=now) & (data_Materias["Cad./FPC"]>=month_vencido)
data_Materias_vencidos=data_Materias.loc[mask_vencido]
#Sort values
data_Materias_vencidos=data_Materias_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_Materias_vencidos=data_Materias_vencidos.reset_index(drop=True)
data_Materias_vencidos["Cad./FPC"]=data_Materias_vencidos["Cad./FPC"].dt.date   #convet datetime to date 

In [27]:

mask=(data_Materias["Cad./FPC"]>=now) & (data_Materias["Cad./FPC"]<=month)
data_Materias=data_Materias.loc[mask]
data_Materias=data_Materias.sort_values(by=["Cad./FPC","Libre utilización"]).reset_index(drop=True)
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].dt.date

#Close Session
#SAP_GUI.Close_session(Iniciar)


"""------------DATAFRAME MATERIAS PARA INFORME-----------------------------------------------------------------------"""
data_Materia_informe=data_Materias[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]

data_Materia_informe_vencidos=data_Materias_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote', 'UMB',
       'Cad./FPC']]

In [28]:
data_Materia_informe_vencidos

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Cad./FPC
0,C041,MP. Natura,PB0077384,357686 AVOBENZONE - DSM,C041,1000.0,1001.0,10893581.0,KG,2023-06-22
1,C026,MP. Unilever,U121460,CLORO METIL ISO TIAZOLINONA,C026,1000.0,1001.0,10891409.0,KG,2023-06-22
2,C274,MP.BEIERSDORF,0018516900,EUTANOL G,C274,1000.0,1001.0,10885537.0,G,2023-06-21
3,C094,MP.Omnilife,PB0050484,BEAUPLEX VH,C094,1000.0,1001.0,10816967.0,KG,2023-06-20
4,C041,MP. Natura,PB0077519,50276607 ESSEN. AROM. LN02 NATURA,C041,1000.0,1001.0,10868981.0,KG,2023-06-20
5,C094,MP.Omnilife,PB0050317,PHYTEXCELL GINSENG,C094,1000.0,1001.0,10800248.0,KG,2023-06-19
6,C274,MP.BEIERSDORF,PB0084144,SODIUM ASCORBYL PHOSPHATE12084,C274,1000.0,1001.0,10875220.0,KG,2023-06-19
7,C221,MP.Jerónimo Martens,100RX01920,50038722 DMDM HYDANTOIN,C221,1000.0,1001.0,10874196.0,KG,2023-06-19
8,C041,MP. Natura,PB0091407,50332025F. FAST E FURIOUS1063GB 5987287,C041,1000.0,1001.0,10873611.0,KG,2023-06-17
9,C011,MP. Prebel,PB0061408,MELALEUCA ALTERNIFOLIA (TEA TREE) OIL,C011,1000.0,1001.0,10834478.0,KG,2023-06-16


In [64]:
"""-----------------------------GENERO SUBTOTALES------------------------------------------------------------"""

data_Materia_Libre=Clean_column_number(data_Materias["Valor libre util."]).sum()
data_Materia_Calidad=Clean_column_number(data_Materias['Valor en insp.cal.']).sum()
Total_Materia=data_Materia_Libre+data_Materia_Calidad
Total_Materia="${:,.2f}".format(Total_Materia)

In [65]:
Total_Materia

'$2,854,011,185.00'

In [66]:
data_Materias

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Cad./FPC
0,C026,MP. Unilever,U121460,CLORO METIL ISO TIAZOLINONA,C026,1000.0,1001.0,10891409.0,KG,1.088,0,0,0,2023-06-22
1,C041,MP. Natura,PB0077384,357686 AVOBENZONE - DSM,C041,1000.0,1001.0,10893581.0,KG,10.200,0,"924,333",0,2023-06-22
2,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10873278.0,G,74,0,0,0,2023-06-23
3,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10883672.0,G,87,0,0,0,2023-06-23
4,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10888146.0,G,136,0,0,0,2023-06-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,C026,MP. Unilever,U28195,MONOESTEARATO DE GLICERILO,C026,1000.0,1001.0,10898733.0,KG,745.100,0,0,0,2023-12-16
452,C274,MP.BEIERSDORF,PB0085762,METHYLPARABEN/737,C274,1000.0,1001.0,10834883.0,KG,45.755,0,"1,491,051",0,2023-12-17
453,C041,MP. Natura,PB0090145,50373100 FRAG COCKTAILIFRAAR 264586,C041,1000.0,1001.0,10895153.0,KG,164.274,0,"24,212,345",0,2023-12-17
454,C329,MP.MILAGROS,PB0092451,ÁCIDO GLUTAMICO,C329,1000.0,1001.0,10884443.0,KG,85.300,0,"2,431,050",0,2023-12-18


In [67]:
"""Send email"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \
        .highlight_between(subset=["Cad./FPC"],color='#FF5733',left=now.date(),right=(now.date()+timedelta(days=7))) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>

    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Los graneles a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

html2="""
    <h2 style="text-align: center"> REPORTE CADUCIDAD DE MATERIAS PRIMAS</h2>
    <p> Por medio del presente informe se evidencia las materias primas segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>
    
    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Las materias primas proximas a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

Send = style_df(data_Graneles_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe)     #Style between LI and LS



# Definir el diccionario de formato
try:
    formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}
    formato_material= {'Ce.': '{:.0f}',"Alm.":'{:.0f}',"Lote":'{:.0f}'}
    # Aplicar el formato a la columna 'Altura'
    Send = Send.format(formato_Graneles)
    Send2=Send2.format(formato_material)
except:
    None

send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")
send_emails(Send2.to_html(),Total_Materia,emails=correos,htmlbody=html2,subject="REPORTE MATERIAS PRIMAS PROXIMOS A VENCER")

In [69]:
send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")

In [70]:
"""-------------------------------------Send email material y granel vencidos--------------------------------"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles y materias primas vencidas, para su correspondiente análisis. </p>

    <h2 style="color: black;" > GRANELES VENCIDOS</h4>

    <div"> {0} </div>

    <h2 style="color: black;" > MATERIA PRIMA VENCIDA </h4>
    

    <div"> {1} </div>


    <h4> Anticipo sinceros agradecimientos. </h4>
 """
formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}


Send = style_df(data_Graneles_Vencidos_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe_vencidos)     #Style between LI and LS
Send=Send.format(formato_Graneles)
Send2=Send2.format(formato_Graneles)
send_emails(Send.to_html(),Send2.to_html(),emails=correos,htmlbody=html,subject="REPORTE GRANELES Y MARIA PRIMA VENCIDO")
